# OpenSearch 인덱스 테스트

이 노트북은 이전에 생성된 오픈 서치 인덱스를 테스트 합니다. 아래의 4가지 방법을 해봅니다.
- Lexical Search
- Sematic Search
- Hybrid Search
- Reranker + Hybrid Search

# 1. 환경 세팅

In [20]:
%load_ext autoreload
%autoreload 2

import sys, os

local_module_path = "../"
sys.path.append(os.path.abspath(local_module_path))
print("local_module_path: ",os.path.abspath(local_module_path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
local_module_path:  /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/03_reranker_hybrid_search


## Bedrock Client 생성
### 참고
- 아래의 노트북은 베드락의 사용을 초기화 합니다. 추후에 베드락의 임베딩 모델등을 사용하기 위함 입니다.



In [21]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from local_utils import bedrock, print_ww
from local_utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())


Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
############### bedrock info
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## Embedding 모델 선택

In [22]:
from local_utils.rag import (
    KoSimCSERobertaContentHandler, 
    SagemakerEndpointEmbeddingsJumpStart,
    get_embedding_model
)

#### [중요] is_KoSimCSERobert == True 경우,  endpoint_name 을 꼭 넣어 주세요.

In [23]:
 %store -r koSimCSE_endpoint_name
print("kosimcse_endpoint_name: \n", koSimCSE_endpoint_name)

kosimcse_endpoint_name: 
 KoSimCSE-roberta-2024-02-12-06-54-55


In [24]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = koSimCSE_endpoint_name
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, boto3_bedrock, aws_region, endpont_name)    

####################
model_name:  Titan-Embeddings-G1
Bedrock Embeddings Model Loaded


# 2. OpenSearch Client 생성
    
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.


In [25]:
from local_utils.proc_docs import get_parameter

In [26]:
aws_region = 'us-east-1'
ssm = boto3.client("ssm", aws_region)

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'lec_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'lec_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'lec_opensearch_password',
)

http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password

In [27]:
from local_utils.opensearch import opensearch_utils

In [28]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

# 3. OpenSearch Index 확인


In [29]:
%store -r index_name
print("index_name: ", index_name)

index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
    print("Index exists and is ready: ")


index_name:  search_retail_demo_store
index_name=search_retail_demo_store, exists=True
Index exists and is ready: 


## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [30]:
from langchain.vectorstores import OpenSearchVectorSearch

In [31]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

# 4. 검색 테스트

## Lexical Search

In [32]:
def opensearch_pretty_print_documents(response):
    '''
    OpenSearch 결과인 LIST 를 파싱하는 함수
    '''
    for doc, score in response:
        print(f'\nScore: {score}')
        # print(f'Document Number: {doc.metadata["row"]}')

        # Split the page content into lines
        lines = doc.page_content.split("\n")
        metadata = doc.metadata
        print(lines)
        print(metadata)        


In [46]:
from local_utils.rag import multiple_search

qry = '남성 정장 신발'

search_result = multiple_search(
    query=qry,
    vector_db= vector_db,
    k = 3,
    h_k = 1, 
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": "[]"}},
        {"term": {"metadata.source": "[]"}},
    ],
    Lexical_Search = True,    
    Semantic_Search = False,    
    Hybrid_Search = False,     
    minimum_should_match = 50,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights= [0.50, 0.50], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)

# opensearch_pretty_print_documents(search_result)

Lexical_Search:  True
Semantic_Search:  False
Hybrid_Search:  False
Query: 
 남성 정장 신발
lexical search query: 
{'query': {'bool': {'filter': [],
                    'must': [{'match': {'text': {'minimum_should_match': '50%',
                                                 'operator': 'or',
                                                 'query': '남성 정장 신발'}}}]}},
 'size': 3}
##############################
Lexcial_Search
##############################

Score: 1.0
['남성용 갈색 가죽 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 10, 'intent': '신발|정장', 'product': '브라운 가죽 신발', 'desc': '남성용 갈색 가죽 정장 신발 한 켤레', 'timestamp': 1707728665.7180805, 'id': '108005eb-897f-4073-ba15-db55c4b127bf'}

Score: 1.0
['남성용 황갈색 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 419, 'intent': '신발|정장', 'product': '탠 슈즈', 'desc': '남성용 황갈색 정장 신발 한 켤레', 'timestamp': 1707728665.7309787, 'id': 'f43f20f6-dfc3-47da-b51a-e940a8f804a9'}

Score: 0.9763395194372438
['남성용 갈색 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 160, '

## Semantic Search

In [47]:
qry = '남성 정장 신발'

search_result = multiple_search(
    query=qry,
    vector_db= vector_db,
    k = 3,
    h_k = 1, 
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": "[]"}},
        {"term": {"metadata.source": "[]"}},
    ],
    Lexical_Search = False,    
    Semantic_Search = True,    
    Hybrid_Search = False,     
    minimum_should_match = 10,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights= [0.51, 0.49], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)

# opensearch_pretty_print_documents(search_result)

Lexical_Search:  False
Semantic_Search:  True
Hybrid_Search:  False
Query: 
 남성 정장 신발
##############################
Semantic_Search
##############################

Score: 1.0
['남성용 갈색 가죽 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 10, 'intent': '신발|정장', 'product': '브라운 가죽 신발', 'desc': '남성용 갈색 가죽 정장 신발 한 켤레', 'timestamp': 1707728665.7180805}

Score: 0.9940967923037008
['남성용 갈색 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 160, 'intent': '신발|정장', 'product': '브라운 슈즈', 'desc': '남성용 갈색 정장 신발 한 켤레', 'timestamp': 1707728665.7221622}

Score: 0.9889926181961841
['남성용 황갈색 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 419, 'intent': '신발|정장', 'product': '탠 슈즈', 'desc': '남성용 황갈색 정장 신발 한 켤레', 'timestamp': 1707728665.7309787}


## Hybrid Search

In [48]:
qry = '남성 정장 신발'

search_result = multiple_search(
    query=qry,
    vector_db= vector_db,
    k = 3,
    h_k = 3, 
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": "[]"}},
        {"term": {"metadata.source": "[]"}},
    ],
    Lexical_Search = False,    
    Semantic_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 10,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights= [0.49, 0.51], # 시멘트 서치에 가중치 0.51 , 키워드 서치 가중치 0.49 부여.
    verbose=True
)

# opensearch_pretty_print_documents(search_result)

Lexical_Search:  False
Semantic_Search:  False
Hybrid_Search:  True
Query: 
 남성 정장 신발
##############################
Lexcial_Search
##############################

Score: 1.0
['남성용 갈색 가죽 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 10, 'intent': '신발|정장', 'product': '브라운 가죽 신발', 'desc': '남성용 갈색 가죽 정장 신발 한 켤레', 'timestamp': 1707728665.7180805, 'id': '108005eb-897f-4073-ba15-db55c4b127bf'}

Score: 1.0
['남성용 황갈색 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 419, 'intent': '신발|정장', 'product': '탠 슈즈', 'desc': '남성용 황갈색 정장 신발 한 켤레', 'timestamp': 1707728665.7309787, 'id': 'f43f20f6-dfc3-47da-b51a-e940a8f804a9'}

Score: 0.9763395194372438
['남성용 갈색 정장 신발 한 켤레']
{'source': 'train.json', 'seq_num': 160, 'intent': '신발|정장', 'product': '브라운 슈즈', 'desc': '남성용 갈색 정장 신발 한 켤레', 'timestamp': 1707728665.7221622, 'id': 'd77642ec-951d-4758-b216-d11b71e69e14'}
##############################
Semantic_Search
##############################

Score: 1.0
['남성용 갈색 가죽 정장 신발 한 켤레']
{'source': 'train.json', 'seq_

## Reranker

In [49]:
%store -r reranker_endpoint_name
print("reranker_endpoint_name: ", reranker_endpoint_name)

reranker_endpoint_name:  Ko-Reranker-2024-02-12-06-58-12


In [54]:
import json
import boto3
from local_utils.reranker import RunReranker


runtime_client = boto3.Session().client('sagemaker-runtime')

qry = '남성 정장 신발'

search_hybrid_result = multiple_search(
    query=qry,
    vector_db= vector_db,
    k=3,
    h_k =3, 
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": "[]"}},
        {"term": {"metadata.source": "[]"}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 10,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.49, .51], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=False
)    

intent = RunReranker(query=qry, query_pair=search_hybrid_result, 
                    boto3_client=runtime_client, 
                    endpoint_name=reranker_endpoint_name, verbose=True)          


### Query: 
 남성 정장 신발
### Reranker payload: 
 {
    "inputs": [
        {
            "text": "남성 정장 신발",
            "text_pair": "남성용 갈색 가죽 정장 신발 한 켤레"
        },
        {
            "text": "남성 정장 신발",
            "text_pair": "남성용 황갈색 정장 신발 한 켤레"
        },
        {
            "text": "남성 정장 신발",
            "text_pair": "남성용 갈색 정장 신발 한 켤레"
        }
    ]
}
### Reranker output: 
 [{'label': 'LABEL_0', 'score': 0.9924402236938477}, {'label': 'LABEL_0', 'score': 0.9903420209884644}, {'label': 'LABEL_0', 'score': 0.9864288568496704}]
### Reranker scorelist: 
 [0.9924402236938477, 0.9903420209884644, 0.9864288568496704]
### The index of the maximum number is: 0
### Highest query: 
 남성용 갈색 가죽 정장 신발 한 켤레
### Highest intent: 
 신발|정장


In [55]:
intent.verbose = False
intent.show_ranked_answers()

#############################################
Query:  남성 정장 신발
#############################################
Re-Ranked Answers: 
Product, Desc, Category, Similar Score:
브라운 가죽 신발 ,  남성용 갈색 가죽 정장 신발 한 켤레 ,  신발|정장 ,  0.99244
탠 슈즈 ,  남성용 황갈색 정장 신발 한 켤레 ,  신발|정장 ,  0.990342
브라운 슈즈 ,  남성용 갈색 정장 신발 한 켤레 ,  신발|정장 ,  0.986429


### 다른 예시

In [74]:
import json
import boto3
from local_utils.reranker import RunReranker


runtime_client = boto3.Session().client('sagemaker-runtime')

qry = '거실 테이블 추천해줘'

search_hybrid_result = multiple_search(
    query=qry,
    vector_db= vector_db,
    k=3,
    h_k =3, 
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": "[]"}},
        {"term": {"metadata.source": "[]"}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 10,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.49, .51], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=False
)    

intent = RunReranker(query=qry, query_pair=search_hybrid_result, 
                    boto3_client=runtime_client, 
                    endpoint_name=reranker_endpoint_name, verbose=True)          


### Query: 
 거실 테이블 추천해줘
### Reranker payload: 
 {
    "inputs": [
        {
            "text": "거실 테이블 추천해줘",
            "text_pair": "방을 돋보이게 하는 펑키 탠 액센트 테이블"
        },
        {
            "text": "거실 테이블 추천해줘",
            "text_pair": "스웰 골든 액센트 테이블로 방을 멋지게 꾸며보세요"
        },
        {
            "text": "거실 테이블 추천해줘",
            "text_pair": "하루를 활기차게 만들어줄 그루비 그레이 커피 테이블"
        }
    ]
}
### Reranker output: 
 [{'label': 'LABEL_0', 'score': 0.06231135502457619}, {'label': 'LABEL_0', 'score': 0.1321992725133896}, {'label': 'LABEL_0', 'score': 0.11836797744035721}]
### Reranker scorelist: 
 [0.06231135502457619, 0.1321992725133896, 0.11836797744035721]
### The index of the maximum number is: 1
### Highest query: 
 스웰 골든 액센트 테이블로 방을 멋지게 꾸며보세요
### Highest intent: 
 가구|테이블


In [75]:
intent.verbose = False
intent.show_ranked_answers()

#############################################
Query:  거실 테이블 추천해줘
#############################################
Re-Ranked Answers: 
Product, Desc, Category, Similar Score:
골든 액센트 테이블 ,  스웰 골든 액센트 테이블로 방을 멋지게 꾸며보세요 ,  가구|테이블 ,  0.132199
그레이 커피 테이블 ,  하루를 활기차게 만들어줄 그루비 그레이 커피 테이블 ,  가구|테이블 ,  0.118368
탠 액센트 테이블 ,  방을 돋보이게 하는 펑키 탠 액센트 테이블 ,  가구|테이블 ,  0.062311
